In [ ]:
%%capture
import os
import site
os.sys.path.insert(0, '/home/schirrmr/code/reversible/reversible2/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')
os.sys.path.insert(0, '/home/schirrmr/code/explaining/reversible//')
%cd /home/schirrmr/


%load_ext autoreload
%autoreload 2
import numpy as np
import logging
log = logging.getLogger()
log.setLevel('INFO')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'png'
matplotlib.rcParams['figure.figsize'] = (12.0, 1.0)
matplotlib.rcParams['font.size'] = 14
import seaborn
seaborn.set_style('darkgrid')

from reversible.sliced import sliced_from_samples

from numpy.random import RandomState

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import copy
import math

import itertools
from reversible.plot import create_bw_image
import torch as th
from braindecode.torch_ext.util import np_to_var, var_to_np
from reversible.revnet import ResidualBlock, invert, SubsampleSplitter, ViewAs, ReversibleBlockOld
from spectral_norm import spectral_norm
from conv_spectral_norm import conv_spectral_norm

def display_text(text, fontsize=18):
    fig = plt.figure(figsize=(12,0.1))
    plt.title(text, fontsize=fontsize)
    plt.axis('off')
    display(fig)
    plt.close(fig)

In [ ]:

rng = RandomState(39482014)
plt.plot(np.sin(np.linspace(0, 2*np.pi, endpoint=False, num=8)))
plt.plot(np.sin(np.linspace(0, 4*np.pi, endpoint=False, num=8)))
a = np.sin(np.linspace(0, 2*np.pi, endpoint=False, num=8))

amps = np.abs(rng.randn(320) + 2)

t_amps = np_to_var(amps, dtype=np.float32).cuda()
signals = amps[:,None] * a[None]
inputs = [np_to_var(signals[:,None,:,None], dtype=np.float32).cuda()]

In [ ]:
def rev_block(n_c, n_i_c):
     return ReversibleBlockOld(
        nn.Sequential(
        nn.Conv2d(n_c // 2, n_i_c,(3,1), stride=1, padding=(1,0),bias=True),
        nn.ReLU(),
            nn.Conv2d(n_i_c, n_c // 2,(3,1), stride=1, padding=(1,0),bias=True)),
         
        nn.Sequential(
        nn.Conv2d(n_c // 2, n_i_c,(3,1), stride=1, padding=(1,0),bias=True),
        nn.ReLU(),
            nn.Conv2d(n_i_c, n_c // 2,(3,1), stride=1, padding=(1,0),bias=True))
    )
    
def res_block(n_c, n_i_c):
     return ResidualBlock(
        nn.Sequential(
        nn.Conv2d(n_c, n_i_c, (3,1), stride=1, padding=(1,0),bias=True),
        nn.ReLU(),
            nn.Conv2d(n_i_c, n_c, (3,1), stride=1, padding=(1,0),bias=True)),
    )

In [ ]:
from discriminator import ProjectionDiscriminator
from reversible.revnet import SubsampleSplitter, ViewAs
from reversible.util import set_random_seeds
from reversible.revnet import init_model_params
from torch.nn import ConstantPad2d
import torch as th
from conv_spectral_norm import conv_spectral_norm
from disttransform import DSFTransform


set_random_seeds(2019011641, True)
feature_model = nn.Sequential(
    SubsampleSplitter(stride=[2,1],chunk_chans_first=False),
    rev_block(2,32),
    rev_block(2,32),
    SubsampleSplitter(stride=[2,1],chunk_chans_first=False),
    rev_block(4,32),
    rev_block(4,32),
    SubsampleSplitter(stride=[2,1],chunk_chans_first=False),
    rev_block(8,32),
    rev_block(8,32),
    ViewAs((-1,8,1,1), (-1,8))
)
feature_model.cuda()


from braindecode.torch_ext.util import set_random_seeds
adv_model = nn.Sequential(
    nn.Conv2d(1,16, (3,1), stride=1, padding=(1,0),bias=True),
    res_block(16,32),
    res_block(16,32),
    nn.AvgPool2d((2,1)),
    res_block(16,32),
    res_block(16,32),
    nn.AvgPool2d((2,1)),
    res_block(16,32),
    res_block(16,32),
    ViewAs((-1,16,2,1), (-1,32)),
    )


adv_model = ProjectionDiscriminator(adv_model,32,2,)

adv_model.cuda()


from reversible.training import hard_init_std_mean
n_dims = inputs[0].shape[1]
n_clusters = 1
means_per_cluster = [th.autograd.Variable(th.ones(n_dims).cuda(), requires_grad=True)
                     for _ in range(n_clusters)]
# keep in mind this is in log domain so 0 is std 1
stds_per_cluster = [th.autograd.Variable(th.zeros(n_dims).cuda(), requires_grad=True)
                    for _ in range(n_clusters)]


for i_class in range(n_clusters):
    this_outs = feature_model(inputs[i_class])
    
    means_per_cluster[i_class].data = th.mean(this_outs, dim=0).data
    stds_per_cluster[i_class].data = th.log(th.std(this_outs, dim=0)).data
    # override phase
    means_per_cluster[i_class].data[len(stds_per_cluster[i_class])//2:] = 0
    stds_per_cluster[i_class].data[len(stds_per_cluster[i_class])//2:] = 0


from copy import deepcopy
optimizer = th.optim.Adam(
                          [
    {'params': list(feature_model.parameters()),
    'lr': 1e-3,
    'weight_decay': 0},], betas=(0,0.9))

optim_dist = th.optim.Adam(
                          [
    {'params': means_per_cluster + stds_per_cluster,
    'lr': 1e-2,
    'weight_decay': 0},], betas=(0,0.9))

optim_adv = th.optim.Adam([{
    'params': adv_model.parameters(),
    'lr': 4e-3, 'weight_decay': 0.00}],#lr 0.0004
                         betas=(0,0.9))

In [ ]:

from reversible.gaussian import get_gauss_samples
from reversible.uniform import get_uniform_samples
from ampphase import (amp_phase_to_x_y, get_amp_phase_samples, amp_phase_sample_to_x_y,
                      outs_to_amp_phase, to_amp_phase,
                     switch_to_other_class)

from reversible.gaussian import get_gauss_samples
from reversible.uniform import get_uniform_samples
from reversible.revnet import invert 
import pandas as pd
from gradient_penalty import gradient_penalty
import time


df = pd.DataFrame()
g_loss = np_to_var([np.nan],dtype=np.float32)
g_grad = np.nan
d_loss = np_to_var([np.nan],dtype=np.float32)
d_grad = np.nan
gradient_loss = np_to_var([np.nan],dtype=np.float32)

In [ ]:
n_epochs = 1001
for i_epoch in range(n_epochs):
    start_time = time.time()
    optim_adv.zero_grad()
    optimizer.zero_grad()
    optim_dist.zero_grad()
    outs = feature_model(inputs[0])
    mean_out = th.mean(outs, dim=1)
    demeaned_out = outs - mean_out.unsqueeze(1)
    new_means = np_to_var(np.abs(rng.randn(320) + 2), dtype=np.float32).cuda()

    remeaned_out = demeaned_out * 0 + new_means.unsqueeze(1)

    inverted = invert(feature_model, remeaned_out)
    y = np_to_var([0]).cuda()
    score_fake = adv_model(inverted, y)
    if (i_epoch % 10) != 0:
        this_inputs = inputs[0]
        score_real = adv_model(this_inputs, y)
        gradient_loss = gradient_penalty(adv_model, this_inputs, inverted[:(len(this_inputs))], y)
        d_loss = -score_real.mean() + score_fake.mean() + gradient_loss * 10
        d_loss.backward()
        d_grad = np.mean([th.sum(p.grad **2).item() for p in adv_model.parameters()])
    else:
        g_loss = -th.mean(score_fake)
        g_loss.backward()
        g_grad = np.mean([th.sum(p.grad **2).item() for p in feature_model.parameters()])
    if (i_epoch % 10) != 0:
            optim_adv.step()
    else:
        optimizer.step()
        optim_dist.step()
    

    
    end_time = time.time()
    epoch_row = {
    'd_loss': d_loss.item(),
    'g_loss': g_loss.item(),
    'grad_loss': gradient_loss.item(),
    #'o_real': th.mean(score_real).item(),
    #'o_fake': th.mean(score_fake).item(),
    'g_grad': g_grad,
    'd_grad': d_grad,
    'runtime': end_time -start_time,}
    df = df.append(epoch_row, ignore_index=True)
    if i_epoch % (max(1,n_epochs // 20)) == 0:
        display_text("Epoch {:d}".format(i_epoch))
        display(df.iloc[-5:])
    
    if i_epoch % (max(1,n_epochs // 20)) == 0:
        fig = plt.figure(figsize=(12,4))
        plt.plot(var_to_np(inverted.squeeze()).T, color='black', lw=0.7);
        display(fig)
        plt.close(fig)
        a = th.mean(outs.detach(), dim=0)
        b = np_to_var(sorted(np.abs(rng.randn(320) + 2)), dtype=np.float32).cuda()#th.linspace(0,10,100).cuda()
        c = a.unsqueeze(0) + b.unsqueeze(1)
        from matplotlib import rcParams, cycler
        inverted = invert(feature_model, c)
        cmap = plt.cm.coolwarm
        N = len(c)
        with plt.rc_context({'axes.prop_cycle': cycler(color=cmap(np.linspace(0, 1, N)))}):
            fig = plt.figure(figsize=(12,4))
            plt.plot(var_to_np(inverted).squeeze().T);
            display(fig)
            plt.close(fig)

In [ ]:
plt.plot(var_to_np(demeaned_out).T);

In [ ]:
# use mean on real data, just out of interest
# 